# BEAT-PD Challenge

Challenge website : https://www.synapse.org/#!Synapse:syn20825169/wiki/596118

Data information : https://www.synapse.org/#!Synapse:syn20825169/wiki/600405


### Ideas/Doubts [Laureano]

VAD like thing to remove unwanted data?
modified MFCC?
X,Y,Z = relative positions or acceleration?

Imp: Predict per person. Maybe UBM like thing and adapt it

In [26]:
# Import required libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML, display

# Imports for the high pass signal
from scipy.signal import butter, freqz, lfilter

# KFold
from sklearn.model_selection import KFold

# Import required modules
from sklearn.preprocessing import StandardScaler

import os.path

# To write WAV File
from scipy.io.wavfile import write

# To make derivative work on multiple CPUs
from concurrent.futures import ProcessPoolExecutor
from functools import partial

import sys

from sklearn.model_selection import StratifiedKFold

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# import transform_data
from transform_data import *
from create_graphs import *


In [ ]:
# Data paths
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"

# FIXME : Move this to data?
path_save_accelerometer_plots = "/home/sjoshi/codes/python/BeatPD/code/accelerometer_plots/"

possible to have participant characteristics from additional db data? ex https://ieeexplore.ieee.org/abstract/document/7911257

# CIS-PD Database

Contains 16 subject_id (patients) for the training set 

- Gender: 11 Male, 5 Female 
- Race: 15 White, 1 NA
- Ethnicity: 15 Not Hispanic or Latino, 1 Unknown
- Age average (standard deviation) : 62.8125 (10.8579)

### CIS-PD: Basic accelerometer visualization

In [ ]:
data_type = "cis"

# TODO: explain
path_train_data, df_train_label = define_data_type(data_type=data_type, data_dir=data_dir)
# display(df_train_label)
# List of interesting measurement id we want to look at
# list_measurement_id=[#'ab5287f4-8261-47ad-8ff2-22b5fe5d246e',
#'db2e053a-0fb8-4206-891a-6f079fb14e3a']#,
# 'ef5b1267-c212-46c5-aab0-4f4437bc6c67',
# '4ec74fb9-7347-435d-83dc-79ad74c3bc49',
# '8e8539ad-8841-476b-b15c-888ce3461989',
# '22b88456-fe8f-4138-af55-be12afca4b81',
# 'ad84583d-e5ae-4926-b077-531a0f7d08a9',
# 'eef56825-940a-4c3e-aebb-60838d60869e',
# 'e0441156-c4b8-467c-8f4f-3b532d594d8f',
# '464ac314-6c4b-4c4a-957c-28a2339150d6']

list_measurement_id = [
    "5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a",
    "cc7b822c-e310-46f0-a8ea-98c95fdb67a1",
    "5163afe8-a6b0-4ea4-b2ba-9b4501dd5912",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "2e3a4c9c-ff01-4a28-bfcf-ce9b7633a39d",  # no inactivity should be removed
    "3cf49c01-0499-4bad-9167-67691711204a",  # no inactivity should be removed PAS LA??
    "3d0f965c-9d72-43d1-9369-1ea3acf963cc",  # PAS LA ???
    "4b269cc2-8f0c-4816-adbf-10c0069b8833",
    "4bc51b90-bfce-4231-85e1-5de3b4bc0745",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
]  # bit of inactivity in the middle]

list_measurement_id = [
    "2d852742-10a9-4c56-9f38-779f2cd66879",
    "4fc3c295-857f-4920-8fa5-f21bfdc7ab4f",
    "db2e053a-0fb8-4206-891a-6f079fb14e3a",
]

# Filter df_train_label according to the measurement_id we are most interested in
df_train_label = interesting_patients(
    df_train_label=df_train_label, list_measurement_id=list_measurement_id
)

# Display filtered df_train_label
display(df_train_label)

# path_no_inactivity_data = remove_inactivity_pct_change(df_train_label)

# Plot the accelerometer data
plot_accelerometer(df_train_label,
                   data_type=data_type,
                   path_train_data=path_train_data,
                   path_accelerometer_plots=path_save_accelerometer_plots
)

### CIS-PD: Create High Pass Data

In [45]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data ancillary_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

high_pass_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+ \
                data_subset+'.high_pass/'

high_pass_filter(df_train_label, high_pass_path, path_train_data, data_type)

Working on  10005cf2-85cf-47ad-8441-f15fb1509f69
Working on  f25cfad6-1835-4f78-9192-d28581ffa4d8
Working on  5cf1a9ff-b13c-457b-a661-ccd617842027
Working on  f55eac42-c808-447b-9b9b-02c10cee579f
Working on  541740be-1158-4169-becb-5ca4926ecd0c
Working on  78428c75-bb3f-48c9-87f1-fe7e9d9f864f
Working on  e9fa3c88-3eba-45a6-93c7-3ef4e5c8f846
Working on  bdda2ebb-1b42-4394-94bd-6994e81c7a09
Working on  415651db-922d-4077-8136-030919aadcd4
Working on  f0719d3c-5203-4ed6-b1b9-448485382ae9
Working on  f8dae5a0-5efa-4ce3-9697-85caafa33acb
Working on  6e5ea446-6c44-4a9d-8ced-c11908938a28
Working on  4f34675f-6ced-4ed4-958e-69719fb4ae6d
Working on  72c3843f-19c9-4f85-bda1-624d684f35d5
Working on  63b2a97b-e4ac-4a24-b68c-1d002a48a111
Working on  5e877b35-38d4-436e-a420-442a9f46dfac
Working on  3b35d3e6-aafa-438e-ace8-0c6ac68780b3
Working on  097e521a-1ca5-4b0d-9eec-5c7fc8ec1a43
Working on  1c14d506-f73d-472e-a7be-5246dd5d26bd
Working on  9f7d2429-c7f7-45b0-a51b-f164817e5f99
Working on  cbf878ff

Working on  13aeee8f-d413-4f13-87aa-e69c55b86e3f
Working on  cd5b4e50-f2ab-42b2-b713-32550ef5a921
Working on  fbfac8ca-2208-4325-9fef-c910fdb4af69
Working on  e10e93d5-d13f-42d9-b552-d5699b159213
Working on  075c4acf-5681-4fa5-9fb4-458d93ae924c
Working on  d6f96c9d-05a2-4590-8c2a-9faf4f4e5f43
Working on  9ea2e6ba-8bda-4154-85e1-a2b99973ae0e
Working on  24557a7b-a693-45c2-9fbe-8c18a02fe8bb
Working on  a6824efb-5d9c-4a0b-b081-4cdde6be23ac
Working on  f2db25a4-88b8-4cd9-ae14-0e1e9396d039
Working on  04dee214-b21b-42a8-b1fa-508c45e4582a
Working on  2a2bc8ca-5531-4dca-a9f7-fcbdce68265e
Working on  f070b9f9-7233-46b2-82cc-d12b4d6ca060
Working on  30c18a31-2941-4b80-8271-1a3be925e38f
Working on  6be93c5a-ad00-41e4-a967-6be1f5ce6fa2
Working on  efbf1a31-63c1-4269-affe-c44cd20f6840
Working on  4ea6e398-f268-4ff1-8790-9f24e2c505d3
Working on  0de346ed-08a3-47c8-85e4-8dcf2ea96b65
Working on  0b775aa9-4d62-428d-8f21-fa24da14eaf9
Working on  ae75b368-454b-41b8-876b-e6c11f54adde
Working on  9ac29d05

Working on  b0fd1f8e-368c-42ab-a6e1-1580257679da
Working on  f03b9f53-9132-480c-8c36-29b2db6d29fa
Working on  3a8fc6b2-335a-44ce-863e-e3218c1a655e
Working on  2aa830f1-84b7-4c8e-a5df-4f97704bd745
Working on  a648d949-7347-478b-9a4c-fc23f3d4d17a
Working on  5baad59e-ba93-4a28-a39c-2e12e2ccfc5e
Working on  acae590a-5c75-4dfa-97ac-8c7dcd949590
Working on  00d5a06e-1719-4d7d-bcbc-c63991a5b1cd
Working on  1205a4e8-1ca0-437f-9cb7-df04e9b18b69
Working on  1cd368b3-5052-4a29-9a1f-630c97522a32
Working on  6deebc3c-d168-4e30-9474-b1cdba6ea826
Working on  a4172c4b-0611-477c-a064-4b4a2e662430
Working on  c5528c15-8b36-40a1-9399-5d06e3605adb
Working on  e92949ae-699a-43b9-8446-f89962be88a6
Working on  dabc0365-44e3-44d4-bc34-eb2c0c1cb287
Working on  8c05ba7a-b893-4d7b-a27c-c10525a31d8f


### CIS-PD: Create Masks for inactivity removal 

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)

print(path_train_data)

In [ ]:
data_type = "cis"

remove_inactivity_highpass(
    df_train_label,
    path_train_data,
    data_type,
    energy_threshold=5,
    duration_threshold=3000,
    plot_frequency_response=False,
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.'+
    data_subset+'.high_pass_mask/')

### CIS-PD: Create first derivative

In [ ]:
data_type = "cis"
# This is only to switch between training_data or ancillary_data which is additional data provided 
data_subset='ancillary_data' #training_data

path_train_data, df_train_label = define_data_type(data_type, data_dir, data_subset)


do_work = partial(
    get_first_derivative, 
    path_train_data=path_train_data,
    derivative_path=data_dir+"cis-pd.training_data.derivative_original_data/",
    padding=True, 
    mask_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.training_data.high_pass_mask/',
)

num_jobs = 1
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

### CIS-PD: Create WAV files - Training Data

#### For Y

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Y

In [ ]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)
do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Y/',
    sAxis='Y',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### For Z

In [ ]:
data_type = "cis"
data_subset='training_data'

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset) 

do_work = partial(
    write_wav, 
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 8
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

#### CIS-PD Write Wav Ancillary Data Z

In [ ]:
data_type = "cis"
data_subset='ancillary_data' 

path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset)

do_work = partial(
    write_wav,
    path_train_data=path_train_data,
    wav_path=data_dir+'cis-pd.'+data_subset+'.wav_Z/',
    sAxis='Z',
    mask_path=data_dir+'cis-pd.'+data_subset+'.high_pass_mask/'
)

num_jobs = 6
with ProcessPoolExecutor(num_jobs) as ex:
    results = list(ex.map(do_work, df_train_label['measurement_id']))

## KFold 

In [ ]:
def get_k_fold(df_train_label,
               n_splits=5,
               subject_id=None,
               data_subset='training_data',
               data_real_subtype=""):
    """
    Function that returns a list of X dataframes (X is according to the number of n_splits chosen)

    The dataframes are the labels needed according to the split 

    Keyword Arguments:
    - df_train_label: Dataframe containing the labels
    - n_split: Optional. The number of folds. Default: 5
    - subject_id: Optional. Specify a subject_id to get measurement_id only for that subject_id 
    - data_real_subtype: Only for REAL-PD database
    """
    kf = StratifiedKFold(n_splits) # shuffle??? 

    # Building the dataframe to split
    X = []

    # if we want the data split for one specific subject_id
    if subject_id:
        df_train_label_subject_id = df_train_label.groupby("subject_id")
        X = df_train_label_subject_id.get_group(subject_id)
    else:
        # if we want to have all a split for all data no matter the subject_id
        # NOTE: I didn't make sure to have one subject_id represented in both train/test
        for idx in df_train_label.index:
            X.append([df_train_label["measurement_id"][idx]])
        X = pd.DataFrame(X)

    kf.get_n_splits(X)

    # Building lists of df_train_label because we have by default 5 splits,
    # so the lists will contain 5 DataFrames with different split indices required
    list_df_train_label = list()
    list_df_test_label = list()
    split_idx = 0
    
    # Removing NaN values only for the purpose of kf.split as stratified kfold don't like NaN values
#     print(type(X))
#     print('Before contains NaN ? : ', X.isnull().sum())
    X = prepro_missing_values(X)
#     print('After contains NaN ? : ', X_no_nan.isnull().sum())
    
    if subject_id == 1046:
        # 1046 is balanced on tremor because that's the only labels we have for that patient
        y = X.iloc[:,-1]
    else:
        # We do the KFOLDs on a balanced on/off only 
        y = X.iloc[:,-3] 

    print("----- " + str(subject_id) + " -----")
    for train_index, test_index in kf.split(X, y):
        print(len(X))
        print(len(df_train_label))
        df_train_label = X.iloc[train_index]
        df_test_label = X.iloc[test_index]
        
        # Following is just to see if the splits are balanced or not 
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_0, train_1, train_2, train_3, train_4 = len(train_y[train_y==0]), \
                                                        len(train_y[train_y==1]), \
                                                        len(train_y[train_y==2]), \
                                                        len(train_y[train_y==3]), \
                                                        len(train_y[train_y==4])
        test_0, test_1, test_2, test_3, test_4  = len(test_y[test_y==0]), \
                                                  len(test_y[test_y==1]), \
                                                  len(test_y[test_y==2]), \
                                                  len(test_y[test_y==3]), \
                                                  len(test_y[test_y==4])
        print('>Train: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d Test: 0=%d, 1=%d, 2=%d, 3=%d, 4=%d' % (train_0, train_1, train_2, train_3, train_4, \
                                                        test_0, test_1, test_2, test_3, test_4))
        
        list_df_train_label.append(df_train_label)
        list_df_test_label.append(df_test_label)

        # name of the file according to its database and type
        # NOTE: Be careful that the end of the name of the folder where to save the kfolds is hardcoded here
        path_save_k_fold_dataframes = (
            data_dir + data_type + "-pd."+data_subset+".k_fold_v2/" + data_real_subtype + "/"
        )
        df_train_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_train_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        df_test_label.to_csv(
            path_save_k_fold_dataframes
            + str(subject_id)
            + "_test_kfold_"
            + str(split_idx)
            + ".csv",
            index=False,
            header=["measurement_id", "subject_id", "on_off", "dyskinesia", "tremor"],
        )
        split_idx = split_idx + 1
    return list_df_train_label, list_df_test_label

### Create the K-Fold files for the CIS database 

In [ ]:
# Define the data type as we have two databases
data_type = "cis"
path_train_data, df_train_label = define_data_type(data_type=data_type)

# Group data by subject_id
df_train_label_subject_id = df_train_label.groupby("subject_id")

data_subset = 'training_data'

# Go through the subject_id and k-fold their data
# FIXME: get_k_fold could me renamed to just create the folds, save them, not return anything
for subject_id, value in df_train_label_subject_id:
    list_df_train_label, list_df_test_label = get_k_fold(
        df_train_label=df_train_label,
        n_splits=5,
        training_or_ancillary=data_subset,
        subject_id=subject_id
    )


### Create the K-Fold Files for the REAL database

In [ ]:
# To generate the files, you have to uncomment one data_real_subtype at a time and
# execute this cell 3 times for the 3 subtypes.

data_type = "real"
data_subset='training_data' # Either training_data or ancillary_data 

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    
    path_train_data, df_train_label = define_data_type(data_type,
                                                   data_dir,
                                                   data_subset,
                                                   data_real_subtype)

    # Group data by subject_id
    df_train_label_subject_id = df_train_label.groupby("subject_id")

    # Go through the subject_id and k-fold their data
    for subject_id, value in df_train_label_subject_id:
        list_df_train_label, list_df_test_label = get_k_fold(
            df_train_label=df_train_label,
            n_splits=5,
            subject_id=subject_id,
            data_subset=data_subset,
            data_real_subtype=data_real_subtype,
        )

# REAL-PD Database

This database, originally named "Parkinson@Home" is renamed to "Real-PD" for this challenge. The study was made over 2 weeks, with at home monitoring. 

The devices used are an android phone, a motorolla watch. 
- `smartwatch_accelerometer` and `smartwatch_gyroscope` : Motorolla Watch
- `smartphone_accelerometer` : Android phone 

-> Question: so is the smartwatch & smartphone accelerometer should measure the same movements? 

The REAL-PD database has many missing values. 

The subject_id `hbv013` is the only one without missing data. Other patients all have at least one missing symptom (`diskenisia`, ) or two (`on/off and tremor`, `on_off and dyskinesia`, `dyskinesia and tremor`) missing.

Measurements id with no data (`on_off`, `dyskinesia` and `tremor` are all missing):
- `b50d1b0c-2cd1-45f8-9097-0742e5cbbcc8`
- `b598c177-4e38-4ea8-8543-bd8f7e580f96`
- `cf841bf8-0082-4ea3-999f-1f43e39a8dc6`
- `b1e15f8a-109f-459b-ba87-46899240ee66`
- `6f0e2580-56ec-4743-9356-d3e4d9a0aee5`
- `773536f6-9b70-43d0-b099-5d167d74924a`
- `54a0e841-ad45-4ba7-ac83-1785c5f7748b`
- `cd9ed2e2-7e04-44c7-b041-7788f133c193`
- `a6954a91-338b-4523-9e4a-5e69a8fac206`

The 3 symptoms are reported as follows in this dataset: 
- `on_off = {0,1}`
  - `Off` : 0 (medication is wearing off) 
  - `On` : 1 (medication is working)
  
- `dyskinesia = {0,1,2}`
  - Without dyskinesia: 0 
  - Non-troublesome dyskinesia: 1 
  - Severe dyskinesia: 2 
  
- `tremor = {0,1,2,3,4}` 
The description of the database mentions `tremor` is rated from 0 to 4 according to its severity, but from all the data, the maximum value of `tremor` recorded is 3. 

Data:
- ancillary
- clinical : UPDRS evaluation score 
- demographics : #TODO


### REAL-PD: Create Masks for inactivity removal for all subtypes

In [46]:
data_type = "real"

for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)
    
    data_subset='training_data' #training_data testing_data
    
    high_pass_path='/home/sjoshi/codes/python/BeatPD/data/BeatPD/real-pd.'+ \
                data_subset+'.high_pass/'+data_real_subtype+'/'
    
    high_pass_filter(df_train_label, high_pass_path, path_train_data, data_type)

data_dir :  /home/sjoshi/codes/python/BeatPD/data/BeatPD/
data_type :  real
data_subset :  ancillary_data
data_real_subtype : smartphone_accelerometer
Working on  2515fa8d-8320-4010-bcd9-5dba52b1b5ba
Skipping 7fde941c-2d8a-4b33-b32d-af016161ee1e as it doesn't exist for the subtype
Skipping 92eea111-b6b0-4d07-a624-cf2b0ecea769 as it doesn't exist for the subtype
Skipping 3f5adb13-512c-4090-96ac-707703077845 as it doesn't exist for the subtype
Working on  42d34bdc-54a9-4b4f-bed6-c02219c6061f
Working on  3bc6b0d1-0204-4a3d-827a-1579a76b411e
Working on  803d4ad6-328d-494f-8923-329175c8b07f
Working on  f6dde26f-279c-4d83-9e5e-8b25cb7e31bd
Working on  ed9d2a14-cb01-42e6-85f0-180b70b2f821
Working on  83cdbcb3-034a-4eb2-9cc6-3757dd4aff22
Working on  2bb369bc-3300-4b73-a57e-58dccee47e07
Skipping 539ab11e-0d9f-4d42-a209-5261a3655275 as it doesn't exist for the subtype
Skipping 475759e6-bdad-44f4-a1be-63d18e475352 as it doesn't exist for the subtype
Working on  d4474cb5-c1c2-497d-921e-3ec7abe02f8

Working on  79320b77-0112-47e6-90e5-020a07a3ffd4
Working on  09920c5d-0c9b-42e9-9b1c-9e853bd4fe50
Working on  f74fdd6f-d28e-4c38-af3d-2149802058ab
Working on  bdc5dc1d-7f8b-445c-8f19-16709dd53351
Working on  0d21a85d-a641-4069-ad2f-7b9d55b63635
Working on  b464bd85-cff0-466f-976f-9ffd29b798f8
Skipping f90d07fa-6362-4d6e-895a-03897208acab as it doesn't exist for the subtype
Working on  3af93fc4-3528-4fa9-8ad5-99f169d6491c
Skipping e479a780-b3fc-44de-b532-d4f3dc7e68fb as it doesn't exist for the subtype
Working on  7202317c-5623-4466-9de1-2042f111dee4
Working on  c7b4e3e2-ef34-4e84-95fa-f86aef445ddd
Working on  b7b33f54-22ed-4d38-88bd-41f31b8a2fdd
Working on  e8180e0a-4e1a-4708-8146-33eb729cf9f5
Skipping eee6e062-9d30-43d3-aa05-3f12f317c12e as it doesn't exist for the subtype
Skipping 2e079684-c0f3-4d3c-9db9-b440ac60e521 as it doesn't exist for the subtype
Working on  3aa9b290-6767-4a07-9693-7985e826371e
Working on  626147c2-5b14-404f-8ef4-59a61d989e8d
Working on  c6fb7516-3c9a-47e0-aed9

Skipping cde8e059-98d7-4864-83e7-692c2f3ba459 as it doesn't exist for the subtype
Working on  65557626-0075-4da9-b740-0e3e389a0df9
Working on  26f49660-ce1a-4946-8f83-f88850f03ec1
Skipping 4c942781-59a6-46e1-95e1-4d74bac4f72d as it doesn't exist for the subtype
Skipping 52cbf184-847c-4c40-8f7d-93abf9b7209d as it doesn't exist for the subtype
Working on  521d28af-2d12-4f47-a6eb-085030391136
Working on  e31c69c0-2820-47c6-afbe-f489e2809b3e
Working on  6545d215-9661-4cca-aceb-f59d52b577c9
Working on  51ecdb9d-bcb9-45fa-8df7-1ea7087e8494
Working on  ad6ac3dc-383a-47d7-8981-d3fa5023ac34
Working on  ecbe8a91-6976-481b-bd3e-501bb7691401
Working on  e40aacd3-0b44-4107-ac51-bd97462d81b2
Skipping 0f9f7bbf-84b2-4fb2-8c5f-e01f495c85b4 as it doesn't exist for the subtype
Working on  bd1b30e9-4aea-4a28-a627-4a7559a258d0
Working on  7909d932-3379-4653-9799-2e372c03daa3
Working on  c9706df3-8d80-479a-af0f-3093a50dea75
Skipping 34d52a27-7dd9-4a68-a989-94811d810839 as it doesn't exist for the subtype
Wo

Working on  523b86de-6271-46f7-9379-500e3e22f5df
Working on  9fa4862a-5b76-410e-afcf-73da4df8a8a5
Skipping 00ec8065-af5a-48be-98e6-a30dfe404747 as it doesn't exist for the subtype
Working on  3e402fec-91f4-46b4-b9e4-27f5c6f08f0e
Skipping dcabef2b-8ee7-4814-b6c3-73c9e5c91f1e as it doesn't exist for the subtype
Skipping 0ed06e7b-fd56-45cd-b287-913439a84b87 as it doesn't exist for the subtype
Skipping 8e741219-dab8-4efa-a8d4-630ec8565d0d as it doesn't exist for the subtype
Working on  6c94b7d8-9be0-4212-80a6-800436b1fd74
Skipping f24c8fcb-1b16-497f-ac12-a938235cc43c as it doesn't exist for the subtype
Skipping 3a18baf4-ddd8-48ad-8419-1a802eef4ecf as it doesn't exist for the subtype
Working on  80d3e287-4c41-4e6e-8ce6-9d1f74de9141
Working on  6f734c22-dc8f-4c99-95c1-d05889ed0746
Working on  8b66a621-86fc-4e55-9b12-c4373770b262
Skipping 5ac3c8f8-c1e0-451b-ba22-ec3c965a604e as it doesn't exist for the subtype
Skipping 627e7fba-82f4-4124-8b3e-9d875c0dbcf4 as it doesn't exist for the subtype
S

Skipping 3051e77d-46ab-44e7-b03d-8b74573c5c97 as it doesn't exist for the subtype
Working on  9446b6f3-6c09-4366-8b60-ac3d10387880
Skipping 80aba7c8-6994-45df-a022-ef61c735c72f as it doesn't exist for the subtype
Working on  d64395cf-19b8-469b-9e3f-f22e60001d8b
Skipping 6caa0bd0-7e68-4281-9dd9-1070786ef026 as it doesn't exist for the subtype
Working on  89c5c34e-bcc4-4079-944e-32617c5f484f
Skipping 763116a6-7386-4dcb-97dc-8cad9a6bb0f2 as it doesn't exist for the subtype
Working on  61cc2c31-f777-4a56-898a-b37b3275ac2c
Working on  439bc1b9-1b23-4691-a454-f4e1c7b8e4d9
Skipping e655453f-9df0-411f-a633-2dd8144d3327 as it doesn't exist for the subtype
Working on  bc47481b-5227-40b8-ac92-4625081d4d52
Working on  9172588d-6be9-4d66-9a78-8c7cb690b29f
Skipping f2256f75-ddbd-49a2-a3de-0069905e11f7 as it doesn't exist for the subtype
Skipping 287ddbef-b7bf-4ff4-b4e9-eaa128bbbe9c as it doesn't exist for the subtype
Working on  fa48f603-41f5-492c-82c6-89f78e6f20b0
Skipping 8c3d0e87-f2c5-4924-ae88-5

...

Working on  df4b6bc8-07aa-4712-a25e-0217c7ba4914
Working on  71fd2ba2-0ba3-4004-980f-d170c2367ed0
Working on  93dcb4f9-0519-476e-bf4d-a18421139245
Working on  fc729282-6bfa-4a75-9aee-1121a473450e
Working on  cca0e216-e011-4775-a3b2-61b1d6188888
Working on  504c4d85-5df7-468d-9f02-120e8a577992
Working on  2fcbce46-3f65-4938-88d0-cbc2f4e8df97
Working on  3520cf42-1958-479f-bdd4-6de926f5c533
Working on  f05a7f8f-e4ac-41bd-9c94-98d1bb661832
Working on  c751bfce-e37d-4930-b8d2-d92bf7004264
Working on  6c8a1135-25db-4af6-a257-9eb3872b53f2
Working on  fc2f7299-08a7-4bf3-b0ed-8f4fa7f1c279
Working on  2ae925c8-bd19-400d-a5bd-3a508643d8ca
Working on  1ee94bbd-caed-4e51-9a2c-7bfc5b20b8a7
Working on  5b234469-216d-44ae-8db3-c4c8ad180231
Working on  517ac84f-7d68-42a5-aedc-f63babedd395
Working on  2089e649-5867-4dc7-96bd-6179cb311752
Working on  42243b7c-714c-4349-8884-88973ddc0849
Working on  377010fc-531b-4647-a760-d0acb40a5fba
Working on  99dffbb5-0cc9-480c-8aa3-6bc90fc3aeb7
Working on  6a2270ac

Working on  734a1e13-f5a2-425f-8540-3e1b6f57fd39
Skipping 658a2398-532b-4d0c-b8fb-67764ec58e9c as it doesn't exist for the subtype
Working on  c15365b5-fa46-45c4-94a1-802ac74bef82
Skipping e429fc2d-08db-4b81-b9c5-865f1fe790eb as it doesn't exist for the subtype
Working on  52eb7473-18f7-477e-ba7b-6723e0bc83c4
Skipping 2898aa29-f72e-4742-bbdc-2d6af815f490 as it doesn't exist for the subtype
Working on  45831d16-3afa-4d54-bc57-03d9608d311f
Skipping 19e35072-c573-4030-a5b3-4b9ff118f99d as it doesn't exist for the subtype
Working on  6b0eabe0-5547-4ef7-8468-7280d28f2675
Skipping 0a58d095-9030-4587-b5a8-359ac80d8813 as it doesn't exist for the subtype
Working on  3c84e8c1-6ff9-4f4b-b3d5-89a5e27c68f4
Skipping 27eccfc4-e329-4695-aee8-6d706b247191 as it doesn't exist for the subtype
Skipping de24ad87-99a0-44f9-a98f-dbb5d5e2873c as it doesn't exist for the subtype
Skipping 38e4c46d-17ac-4445-815f-0489996e80b9 as it doesn't exist for the subtype
Skipping e9e16344-54ee-4fbc-8b5f-0e7565c2de36 as i

Working on  ebd6c616-0e16-4c8c-8358-aeb2e4326764
Working on  afe7209e-ac7d-4be9-b100-decb9e81cd42
Working on  db9b6763-6d0c-4d8a-b27c-104ebec29dd3
Working on  97642041-eeff-409c-b138-ff2ec19af5a2
Working on  1095c912-48ac-44c8-a8a1-0d1c29d0361b
Working on  a5b6a1de-a75b-4169-aab8-79ba10f95f46
Working on  105b7cc6-f580-4849-8e49-7c95463350bd
Working on  bd3defa5-af6a-4510-bc0c-4dbbe87f1559
Working on  75b3cd77-6033-4fa6-9bc6-71e334ee3ade
Working on  08215a48-b258-4e58-9ec4-c33181451c27
Working on  7c840fba-4df1-4fd8-84a9-9fe1447f6173
Working on  d2c18bea-35f3-4cdd-873d-a8fe14754300
Working on  35dc474e-3fdf-41a6-a2c7-21e34a398656
Working on  da9380be-ff18-4067-9fe4-99285dc20232
Working on  24768c72-704a-4f92-ae1f-710731242291
Working on  2aeac34a-6a27-4513-86c8-ac32ba5c8c17
Working on  73b91037-8021-44b7-ade2-d7f7549c0a4d
Working on  7ff5ce84-1c06-49ee-9cf1-68a4fd140332
Working on  6d0572ed-774e-4e17-9857-c0befe8fe0ae
Working on  b78b9824-b309-4a3b-bff3-1544cb6338cd
Working on  b2c577d0

Working on  d25782de-83c0-4f73-adfb-7637d8a7d39a
Working on  b1e15f8a-109f-459b-ba87-46899240ee66
Working on  100d561c-ee09-4b5f-92a3-094faa4b2f7c
Working on  047b34fa-ff8a-4fb9-8eea-7cc0c8f307b2
Working on  50fa963c-7ee8-45a2-8a02-93a00a9cc479
Working on  d343a0d3-6d08-435f-a523-ff16758bda20
Working on  9d97b84a-0d04-4e61-a987-6fe6d1ca72cc
Working on  f5513696-a0e3-4e15-b724-2e4674e15074
Working on  22b2f310-70d9-4a3d-bc6f-c0697d4202d6
Working on  b21fcc94-2257-4399-8572-5ab838a46e90
Working on  b5c5835b-c794-4efe-ac42-17465cb69e31
Working on  c8fc840c-a165-4b18-8458-e82006166b8c
Working on  ec61f4f7-ba05-45aa-bb66-afcfcdd34480
Working on  3eae8ef7-0e6f-45a9-be4c-9e8c3cfc4cf7
Working on  8b0d9a1a-9e36-4453-9df3-eb64ff724da9
Working on  304657b8-92fc-45c6-89c4-37f7cd3b77e4
Skipping bc0a8345-1ef1-4faf-b1b0-41f86d4f1bab as it doesn't exist for the subtype
Working on  7fc930be-9cce-4261-b561-a74d88c44106
Skipping 59a73268-4e1a-4426-bf73-0fb22f58fa1a as it doesn't exist for the subtype
Wor

Working on  2d87ae0b-c2c3-474e-971b-427ff0fb2387
Working on  cd9ed2e2-7e04-44c7-b041-7788f133c193
Skipping 528ddcd4-800d-4b18-a086-74cf26fd9a48 as it doesn't exist for the subtype
Working on  78247cdd-af79-4edc-bf7a-fec6ef295c83
Working on  72af00e8-1dd6-4fcf-a91d-c24445b6e173
Working on  4ee4382e-5e6d-465e-b3b4-2e0fc75a160b
Working on  f7427591-867d-4944-9bee-eb95c673b568
Working on  d74db5cf-c7df-4c5a-bd24-e92eac56d349
Working on  5e8f2606-0d49-4d36-a98c-44bce687866f
Working on  e8a98ea3-1079-48e1-b3ea-3a70d167122e
Skipping a6954a91-338b-4523-9e4a-5e69a8fac206 as it doesn't exist for the subtype
Working on  c825c097-2403-46b5-ab57-ed72f6cd0344
Working on  eef1a89c-8f6a-4be0-b0d9-2a224f253c00
Working on  e7a51859-c9ae-4436-bf69-e9ec0b6fac9e
Working on  5f6ec7b4-2374-4309-9349-14af6d2eebf2
Working on  22b9cab0-aadd-4d9f-9d03-6c85c85be824
Working on  ead7c4df-53fc-493e-be19-39e5c269f003
Working on  f71ba377-2f59-4605-ae3f-8221d2120c7f
Working on  a6a309e7-2450-4ba5-89d3-ce2608989c9a
Wor

In [ ]:
data_type = "real"
#data_real_subtype='smartphone_accelerometer'
#data_real_subtype = "smartwatch_accelerometer"
#data_real_subtype='smartwatch_gyroscope'

# List of interesting measurement id we want to look at
# list_measurement_id = [
#     "5b4c7c81-659d-40ea-a1fd-59622074fd10",
#     "ee053d95-c155-400d-ae42-fe24834ad4a9",
#     "ce51ee31-8553-4321-9f83-8cd3dabe2f66",
#     "e07708ff-7b8d-4070-af70-3aa81423ab5b",
#     #'7d3f4b7a-167f-4a26-9062-94ce9d8794c1',
#     "99af8d14-cd09-4107-9502-355378ba4e08",
#     #'7d5ac31a-cb53-40f7-8188-0b13724ea55c',
#     "9e43840b-dd89-498b-af1a-a62896a4d5d9",
#     "e391f546-bf8a-46c7-a16c-95bc02f40629",
# ]

# list_measurement_id = ['0c310608-1a32-4b09-b164-375d93ddb2aa']

# Filter df_train_label according to the measurement_id we are most interested in
# df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    data_subset='ancillary_data' #training_data testing_data 
    print(data_real_subtype)
    path_train_data, df_train_label = define_data_type(data_type,
                                                       data_dir,
                                                       data_subset,
                                                       data_real_subtype)

#     list_measurement_id = ['f465145e-562d-4c02-b347-bc42ae3b4998']#['33f5a031-43a8-496a-89ee-0b9d99019617', '2515fa8d-8320-4010-bcd9-5dba52b1b5ba']
    # Filter df_train_label according to the measurement_id we are most interested in
#     df_train_label = interesting_patients(df_train_label=df_train_label,
#                                           list_measurement_id=list_measurement_id)

    remove_inactivity_highpass(
        df_train_label,
        path_train_data=path_train_data,
        data_type=data_type,
        energy_threshold=5,
        duration_threshold=3000,
        plot_frequency_response=False,
        plot_accelerometer_after_removal=False,
        mask_path=data_dir+'/real-pd.'+
        data_subset+
        '.high_pass_mask/'+data_real_subtype+'/')

### REAL-PD: Create first derivative for all subtypes 

In [ ]:
for data_real_subtype in ['smartphone_accelerometer','smartwatch_accelerometer','smartwatch_gyroscope']:
    path_train_data, df_train_label = define_data_type(data_type=data_type)
    print(len(df_train_label))
    for idx in df_train_label.index:
        try:
#             print('where we get the file : ', path_train_data)
            df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
        except FileNotFoundError:
            print('Removing ' + df_train_label["measurement_id"][idx] +
                  ' as it doesn\'t exist for ' +
                  data_real_subtype)
            df_train_label = df_train_label.drop(idx)
        print(len(df_train_label))
    do_work = partial(
        get_first_derivative, 
        path_train_data=path_train_data,
        derivative_path="real-pd.training_data.derivative_original_data/"+data_real_subtype+'/',
        padding=True, 
        mask_path=data_dir+'/real-pd.training_data.high_pass_mask/'+data_real_subtype+'/'
    )

    num_jobs = 8
    with ProcessPoolExecutor(num_jobs) as ex:
        results = list(ex.map(do_work, df_train_label['measurement_id']))


### REAL-PD: Create WAV files for all subtypes

### X Axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="X", data_type="real")

# data_type ="real"
# #FIXME add the one missing when it's done running 
# #'smartphone_accelerometer', 'smartwatch_accelerometer', smartwatch_gyroscope
# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset = 'ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type, data_subset, data_real_subtype)
# #     list_mesurement_id=['33f5a031-43a8-496a-89ee-0b9d99019617']
#     # Filter df_train_label according to the measurement_id we are most interested in
# #     df_train_label = interesting_patients(df_train_label=df_train_label, list_measurement_id=list_measurement_id)

#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
        
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'real-pd.'+data_subset+'.wav_X/'+data_real_subtype+'/',
#         sAxis='X',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/'
#     )

#     num_jobs = 6
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

### Y axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Y", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data'#'training_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     for idx in df_train_label.index:
#         try:            
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)

#     do_work = partial(
#         write_wav,
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Y/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Y',
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))

### Z Axis

In [ ]:
data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
create_real_wav_files(data_subset="ancillary_data", data_dir=data_dir, sAxis="Z", data_type="real")


# data_type = "real"

# for data_real_subtype in ['smartwatch_gyroscope']:#['smartphone_accelerometer', 'smartwatch_accelerometer', 'smartwatch_gyroscope']:
#     data_subset='ancillary_data' #training_data
#     path_train_data, df_train_label = define_data_type(data_type,
#                                                    data_dir,
#                                                    data_subset,
#                                                    data_real_subtype)
#     display(df_train_label)
#     for idx in df_train_label.index:
#         try:
#             print(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#             df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
#         except FileNotFoundError:
#             print('Removing ' + df_train_label["measurement_id"][idx] +
#                   ' as it doesn\'t exist for ' +
#                   data_real_subtype)
#             df_train_label = df_train_label.drop(idx)
#     display(df_train_label)
#     do_work = partial(
#         write_wav, 
#         path_train_data=path_train_data,
#         wav_path=data_dir+'/real-pd.'+data_subset+'.wav_Z/'+data_real_subtype+'/',
#         mask_path=data_dir+'/real-pd.'+data_subset+'.high_pass_mask/'+data_real_subtype+'/',
#         sAxis='Z'
#     )

#     num_jobs = 8
#     with ProcessPoolExecutor(num_jobs) as ex:
#         results = list(ex.map(do_work, df_train_label['measurement_id']))
        

# Tests & Drafts, back-up space that's not important, just notes

In [ ]:
# Change the default option to display all row with display(DF)
# pd.set_option('display.max_rows', 1000)

### Tried to do convolution instead of np.multiply and dot product to get the derivative

In [ ]:
df2 = pd.DataFrame(
    np.array(
        [
            [0.2, 1, 5, 9],
            [0.4, 2, 6, 10],
            [0.6, 3, 7, 11],
            [0.8, 4, 8, 12],
            [1, 5, 9, 13],
            [1.2, 6, 10, 14],
            [1.4, 7, 11, 15],
            [1.6, 7, 11, 15],
            [1.8, 8, 12, 16],
            [2, 9, 13, 16],
        ]
    ),
    columns=["Timestamp", "X", "Y", "Z"],
)
display(df2)
m = np.linspace(-3, 3, num=2 * 3 + 1)
display(m)

np.convolve(df2.loc[0:6, "X"], m)

### How to filter a DataFrame

In [ ]:
# Analysis of the data to find edge cases


# Create variable with TRUE if nationality is USA
dys = df_train_label["dyskinesia"] > 1

# Create variable with TRUE if age is greater than 50
tre = df_train_label["on_off"] > 0

# Select all cases where nationality is USA and age is greater than 50
df_train_label[dys & tre]

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)

In [ ]:
# df_zeros = pd.DataFrame([False,True,False,False,False,True,False]).astype(int)

df_zeros = np.array([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=bool)

display(df_zeros.astype(int))
count = 0
duration_threshold = 2
indices_list = []  # List of tuples
howmany = 0
for i in range(0, len(df_zeros)):
    if df_zeros[i] == 1:
        count = count + 1
        print("1 à lindex", i)
    else:
        if count >= duration_threshold:
            print("threshold atteint start ", start, " end at ", end)
            start = i - count
            end = i - 1
            indices_list.append((start, end))
            howmany = howmany + 1
            count = 0
        # if it doesn't reach the threshold, we change the 1 for 0 because we don't want to remove those
        elif count >= 1:
            print("Effacer les 1 de ", start, " a ", end)
            df_zeros[i - count : i] = [0] * (end - start)
            count = 0

display(df_zeros.astype(int))